**ids-pdl12-tut.ipynb**: This Jupyter notebook is provided by Joachim Vogt for the *Python Data Lab* of the module *CH-700 Introduction to Data Science* offered in Fall 2023 at Constructor University. Jupyter notebooks and other learning resources are available from a dedicated *module platform*.

# Statistics in Python

This tutorial introduces Python tools for displaying empirical statistical distributions and their theoretical counterparts as well as hypothesis testing. Follow the instructions below to learn to

- [ ] generate sequences of random numbers using the NumPy module `random`,
- [ ] construct and display empirical distributions in the form of histograms,
- [ ] approximate probability density functions using kernel density estimators,
- [ ] work with important methods of the SciPy module `stats`,
- [ ] understand hypothesis testing and apply $z$ tests and Student's t tests to data,
- [ ] check the normality assumption using hypothesis tests and probability plots.

If you wish to keep track of your progress, you may edit this markdown cell, check a box in the list above after having worked through the respective part of this notebook, and save the file.

*Short exercises* are embedded in this notebook. *Sample solutions* can be found at the end of the document.

## Preparation

Run the following code cell to import standard Python data science libraries. The NumPy module facilitates efficient processing of numerical arrays, and is usually imported as `np`. From the matplotlib library we import the package `pyplot` using the standard abbreviation `plt`. The SciPy module `stats` provides a variety of statistical tools and functions. The magic command `%matplotlib inline` (IPython shell) allows for inline display of graphics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
%matplotlib inline

## Pseudo-random number generators

In statistical data analysis and modeling, pseudo-random number generators are frequently used, e.g., to study statistical operations by means of synthetic measurements (also called surrogate data) drawn from a given distribution. Two important distributions considered here are the uniform distribution and the normal (Gaussian) distribution. Before we look at how they are implemented in Python, let us review a few theoretical concepts.

### Random variables and statistical distributions

Recall that measurements and their errors are modeled by *random processes*, and their outcomes by random variables. For *continuous random variables*, probabilities do not refer to single values but to ranges of values. With the probability to find a continuous random variable $U$ in the range $[a,b]$ denoted as $\mathrm{Prob} (a \le U \le b)$, the *cumulative distribution function (CDF)* $\Phi$ is defined through $\Phi (u) = \mathrm{Prob} (U \le u)$, i.e., it is the probability that the random variable $U$ assumes a value smaller than or equal to the number $u$.

If the cumulative distribution function $\Phi = \Phi(u)$ is a smooth (differentiable) function, then its derivative
$$
p(u) \; = \; \frac{\mathrm{d} \Phi}{\mathrm{d} u} \, = \, \Phi'(u) 
$$
is called *probability density function (PDF)*, but also density, distribution function, or simply distribution.

- The PDF is *non-negative*: $p(u) \ge 0$.
- $\mathrm{Prob} (a \le U \le b) \, = \, \Phi(b) - \Phi(a) \, = \, \int_a^b p(u) \, \mathrm{d} u ~.$
- *Normalization*: $\int_{-\infty}^{\infty} p(u) \, \mathrm{d} u \, = \, 1 ~.$

*Quantiles* divide the range of a distribution into intervals of equal probability, and are obtained as solutions $u$ of $\Phi(u) = q$ with predefined probabilities $q$, thus $u = \Phi^{-1}(q)$. The *quantile function* or *percent point function (PPF)* is the inverse of the CDF.

### Normal distribution

The normal distribution is characterized by two parameters, namely, the mean $u_0$ and the standard deviation $\sigma$. Its PDF is given by
$$
p(u) \, = \, \frac{1}{\sqrt{2\pi\sigma^2}} e^{-(u - u_0)^2/2\sigma^2} ~.
$$
The function `normal()` from the NumPy module `random` generates (pseudo-)random numbers according to the normal distribution, with the mean $u_0$ and the standard deviation $\sigma$ defined through the keyword parameters `loc` and `scale`. A third keyword parameter `size` allows for specifying the length of the shape of the output array.

In [ ]:
print(np.random.normal(loc=10,scale=10,size=(2,5)))

The standard normal distribution has zero mean and unit variance:
$$
p(u) \, = \, \frac{1}{\sqrt{2\pi}} e^{-u^2/2} ~.
$$
Random numbers according to the standard normal distribution are generated conveniently by means of the functions `standard_normal()` or `randn()`, also from the NumPy module `random`.

In [ ]:
print(np.random.standard_normal(5))
print(np.random.randn(5))

### Uniform distribution

Uniform distributions are non-zero and constant in an interval $[a,b]$, and zero otherwise:
$$
p(u) \, = \, 
\left\{ 
\begin{array}{ccl} 
\frac{1}{b-a} & \text{if} & u \in [a,b] ~, \\
0             & \text{if} & u \notin [a,b] ~.
\end{array}
\right.
$$
The function `uniform()` from the NumPy module `random` generates uniform random numbers with the keyword parameters `low` and `high` set to $a$ and $b$, respectively.

In [ ]:
from numpy.random import uniform
print(np.random.uniform(low=10,high=20,size=(2,5)))

The standard uniform distribution yields values in the interval $[a,b] = [0,1]$. Random number generators from the NumPy module `random` for this distribution are `random()`, `random_sample()`, and `rand()`.

In [ ]:
print(np.random.random(5))
print(np.random.random_sample(5))
print(np.random.rand(5))

### Generator instance and seed values

While functions for random number generation can be called in the usual manner as demonstrated above, it is recommended to create random numbers from an instance of a Generator class produced by the constructor function `default_rng()`.

In [ ]:
rng = np.random.default_rng()
print(rng.random(5))
print(rng.standard_normal(5))

The function `default_rng()` accepts a seed parameter to initialize the random number sequence to a controlled state. The same random number sequence may be reproduced for test purposes, as one can see by repeatedly running the following cell.

In [ ]:
seed = 42
rng = np.random.default_rng(seed)
rng.random(5)

## Density estimation

An empirical distribution constructed from the data is a statistical estimator for the PDF of the underlying random process. The procedure of constructing such an empirical distribution is called *density estimation*. A simple and popular density estimator is the *histogram*, collecting data into predefined intervals called *bins*. The resulting occurence frequencies are plotted as a step function using the `hist()` method from `matplotlib.pyplot`. The keyword `density` normalizes the output so that it can be graphically compared to the theoretical PDF. The keyword `bins` gives the number of intervals and thus controls the resolution of the histogram.

In [ ]:
rng = np.random.default_rng(99)
u = rng.standard_normal(1000)
x = np.linspace(-4,4,81)
d = np.exp(-x**2/2)/np.sqrt(2*np.pi)
plt.figure(figsize=(8,4))
plt.plot(x,d,label='Theoretical PDF')
plt.hist(u,bins=20,density=True,label='Empirical PDF')
plt.title('Gaussian distribution and random numbers: histogram')
plt.xlim((-4,4))
plt.grid()
plt.legend()

More refined than histograms are *kernel density estimators (KDEs)* which acculumate contributions from predefined functions (kernels) around the data. KDE implementations exist in several Python packages. Here we choose ``kdeplot()`` from the seaborn module. The type of kernel and the resolution can be controlled by means of the keywords `kernel` and `bw`, respectively. See the documentation for details.

In [ ]:
from seaborn import kdeplot
rng = np.random.default_rng(99)
u = rng.standard_normal(1000)
x = np.linspace(-4,4,81)
d = np.exp(-x**2/2)/np.sqrt(2*np.pi)
plt.figure(figsize=(8,4))
plt.plot(x,d,label='Theoretical PDF')
kdeplot(u,fill=True,label='Empirical PDF')
plt.title('Gaussian distribution and random numbers: KDE plot')
plt.xlim((-4,4))
plt.grid()
plt.legend()

### Exercise: Density estimation

Using the same parameters as for the standard normal distribution example, create a set of random numbers following the standard uniform distribution, then a histogram plot and a KDE plot.

## Random variables as provided by `scipy.stats`

The SciPy module `stats` provides an environment for working with random variables. Consult the [user guide](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html) and the 
[reference manual](https://docs.scipy.org/doc/scipy/reference/reference/stats.html#statsrefmanual) for an introduction to the module.

### Example: Uniform distribution in `scipy.stats`

To demonstrate the attributes and methods associated with the continuous random variables class in the SciPy module `stats`, consider the uniform distribution on an interval $[a,b]$ as implemented in `scipy.stats.uniform()`. The keyword arguments `loc` and `scale` correspond to the startpoint $a$ and the interval length $b-a$, respectively. To obtain an overview of further parameters and methods associated with `scipy.stats.uniform()`, uncomment the line in the cell below.

In [ ]:
#scipy.stats.uniform?

For instance, the support and basic statistics for the default case (default parameters `loc=0` and `scale=1`, support $[0,1]$) can be obtained directly from the corresponding methods.

In [ ]:
print('Support:',scipy.stats.uniform.support())
m,v,s,k = scipy.stats.uniform.stats(moments='mvsk')
print('Moments: m={:.2f}, v={:.4f}, s={:.2f}, k={:.3f}'.format(m,v,s,k))

Alternatively, one may first create a so-called *frozen* (continuous) random variable, and then call suitable methods to compute statistics. This approach is exemplified below, using a shifted (`loc=-1`) and stretched (`scale=2`) version of the standard uniform distribution.

In [ ]:
### Uniform distribution on [0,1]
print('Uniform distribution on [0,1]')
urv1 = scipy.stats.uniform()
print('* Support:',urv1.support())
m,v,s,k = urv1.stats(moments='mvsk')
print('* Moments: m={:.2f}, v={:.4f}, s={:.2f}, k={:.3f}'.format(m,v,s,k))
print()
### Uniform distribution on [-1,1]
print('Uniform distribution on [-1,1]')
uloc = -1
usca = 2
urv2 = scipy.stats.uniform(loc=uloc,scale=usca)
print('* Support:',urv2.support())
m,v,s,k = urv2.stats(moments='mvsk')
print('* Moments: m={:.2f}, v={:.4f}, s={:.2f}, k={:.3f}'.format(m,v,s,k))

Display the probability density functions (PDFs) for both cases.

In [ ]:
x = np.linspace(-2,2,81)
plt.figure(figsize=(6,4))
plt.title('Probability density functions (PDFs)')
plt.plot(x,urv1.pdf(x),'b-',lw=3,label='Uniform on {}'.format(urv1.support()))
plt.plot(x,urv2.pdf(x),'r-',lw=3,label='Uniform on {}'.format(urv2.support()))
plt.xlim([-2,2])
plt.ylim([0.0,1.1])
plt.legend()
plt.grid()

Display the cumulative distribution functions (CDFs) for both cases.

In [ ]:
x = np.linspace(-2,2,81)
plt.figure(figsize=(6,4))
plt.title('Cumulative distribution functions (CDFs)')
plt.plot(x,urv1.cdf(x),'b-',lw=3,label='Uniform on {}'.format(urv1.support()))
plt.plot(x,urv2.cdf(x),'r-',lw=3,label='Uniform on {}'.format(urv2.support()))
plt.xlim([-2,2])
plt.ylim([0.0,1.1])
plt.legend()
plt.grid()

Display the percent point functions (PPFs) for both cases.

In [ ]:
q = np.linspace(0,1,81)
plt.figure(figsize=(6,4))
plt.title('Percent point functions (PPFs)')
plt.plot(q,urv1.ppf(q),'b-',lw=3,label='Uniform on {}'.format(urv1.support()))
plt.plot(q,urv2.ppf(q),'r-',lw=3,label='Uniform on {}'.format(urv2.support()))
plt.xlim([0,1])
plt.ylim([-1,1])
plt.legend()
plt.grid()

### Exercise: Normal distribution in `scipy.stats`

Consider the standard normal distribution and repeat the sequence of steps from the previous example.

### Random number generation in `scipy.stats`

In `scipy.stats` terminology, random numbers are referred to as random variates, and generated by means of the method `rvs()`. In the cell below, a sequence of uniformly distributed random numbers is created. The resulting histogram is displayed against the PDF.

In [ ]:
x = np.linspace(-2,2,81)
plt.figure(figsize=(6,4))
plt.title('Histogram and model distribution')
plt.plot(x,urv2.pdf(x),'r-',lw=3,label='Uniform pdf')
data = urv2.rvs(size=20)
bins = np.linspace(-1,1,5)
plt.hist(data,bins=bins,color='red',\
         alpha=0.3,density=True,label='Random variates')
plt.xlim([-2,2])
plt.legend()
plt.grid()

Interpreting the set of random variates as input data, one can fit the parameters of the uniform distribution.

In [ ]:
loc_est,scale_est = scipy.stats.uniform.fit(data)
print('Estimated location (lower bound of uniform distr. :',loc_est)
print('Estimated scale (upper bound minus lower bound)   :',scale_est)

Initiatizing random number sequences in a reproducible manner is possible also in `scipy.stats`. Consult the documentation on `RandomState` and `Generator` objects in NumPy, and how to import the information in `scipy.stats`. 

## Normally distributed random variables

To prepare for hypothesis testing, let us review and illustrate important properties of the normal (Gaussian) distribution.

A normally distributed random variable $U$ is fully characterized by the mean $\mu = \bar{U}$ and the standard deviation $\sigma = \Delta U$. The probability to find a measurement of $U$ in the range $[\mu-z\sigma,\mu+z\sigma]$ is given by
$$
\gamma \, = \, \mathrm{Prob} (\mu-z\sigma \le U \le \mu + z\sigma) 
\, = \, \Phi (z) - \Phi(-z) 
\, = \, 2 \Phi (z) - 1 
$$
where $\Phi$ denotes the cumulative distribution function of the standard normal distribution with zero mean ($\mu=0$) and unit variance ($\sigma=1$).

Choosing $z=2$ gives $\mathrm{Prob} (\mu-2\sigma \le U \le \mu + 2\sigma) = 0.9545$, hence the probability that a measurement ends up within $\pm 2 \sigma$ around the mean $\mu$ is larger than $95\%$.

In [ ]:
z = 2
gamma = scipy.stats.norm.cdf(z) - scipy.stats.norm.cdf(-z)
print('Probability gamma for z = {:.2f} : gamma = {:.2f}%'.format(z,gamma*100))

One may also ask how many ($z$) standard deviations ($\sigma$) around the mean ($\mu$) must be considered to arrive at a given probability. To this end, one may employ the percent point function $\Phi^{-1}$ (inverse CDF). The number $z$ is then given by
$$
z \; = \; 
\Phi^{-1} \left( \frac{1+\mathrm{Prob} (\mu-z\sigma \le U \le \mu + z\sigma)}{2} \right) 
\; = \; \Phi^{-1} \left( \frac{1+\gamma}{2} \right) ~.
$$

In [ ]:
gamma = 0.95
z = scipy.stats.norm.ppf(0.5*(1+gamma))
print('Score z for probability gamma = {:.2f}% : z = {:.2f}'.format(gamma*100,z))

## Hypothesis testing

*Hypothesis testing* is a method in inferential statistics with the following elements.

- Formulate a *null hypothesis* $H_0$ and (at least implicitly) an alternative hypothesis $H_1$ referring to a testable property of the distribution.
- Define a *test statistic* $T$ sensitive to differences between $H_0$ and $H_1$.
- Choose a *significance level* $\alpha$ (maximum tolerated false positive rate).
- For the chosen test statistic $T$, find the value $t$ from the sample.
- Compute the *probability value* ($p$ value): probability that an empirical estimate of $T$ is at least as extreme as the observed value $t$. 
- The null hypothesis $H_0$ is rejected (in favor of $H_1$) if $p < \alpha$.
- *Type I error*: a true null hypothesis is rejected (false positive).
- *Type II error*: a false null hypothesis is not rejected (false negative).

*Examples of statistical tests*

- $z$ tests: under $H_0$, the test statistic is normally distributed.
- Student's $t$ tests: under $H_0$, the test statistic $T$ follows a Student's $t$ distribution.
- Normality tests: check if a sample is drawn from a normal distribution.

In the examples below, the standard choice for the significance level $\alpha$ is as follows.

In [ ]:
alpha = SignificanceLevel = 0.05

### $z$ tests

This category summarizes statistical hypothesis tests where under the null hypothesis $H_0$, the test statistic $Z$ is normally distributed: $Z \sim \mathcal{N} (\mu,\sigma^2)$. $Z$ tests are often used as so-called location (mean $\mu$) tests when either the scale (standard deviation $\sigma$) is known, or the sample size $N$ is large enough for the exact (Student's $t$) distribution to be well approximated by a normal distribution. The test statistic $Z$ is the scaled difference between the sample mean $\bar{u}$ and $\mu$: 
$$
Z \, = \, \frac{\bar{u}-\mu}{\sigma/\sqrt{N}} \, \sim \, \mathcal{N} (0,1) ~.
$$
Under the null hypothesis $H_0$ it is assumed that the sample mean $\bar{u}$ does not show a significant difference from the true mean $\mu$.

To demonstrate the test procedure, we first generate a sample of normally distributed data.

In [ ]:
### Define the true parameters of the underlying normal distribution.
TrueMean = 3
TrueStd = 5
### Obtain a sample drawn from the underlying normal distribution.
N = SampleSize = 100
data = scipy.stats.norm.rvs(loc=TrueMean,scale=TrueStd,size=SampleSize)

We compute the sample mean $\bar{u}$ and the resulting value $z$ of the test statistic $Z$. Using the cumulative distribution function (CDF) $\Phi$ of the standard normal distribution, we obtain the probability ($p$ value) of getting a value at least as extreme (small or large) as $\bar{u}$ in the theoretical distribution of sample means:
$$
p \, = \, \Phi(-|z|) + \left[ 1 - \Phi(|z|) \right] 
\, = \, 2 \Phi(-|z|) ~.
$$
This kind of test is called *two-tailed* because the rejection region comprises both the left (lower) tail of the distribution (area $\Phi(-|z|)$) and right (upper) tail (area $1 - \Phi(|z|) = \Phi(-|z|)$).

In [ ]:
print('True mean           : {:3f}'.format(TrueMean))
print('Sample size         : {}'.format(SampleSize))
### Compute the sample mean.
SampleMean = np.mean(data)
print('Sample mean         : {:3f}'.format(SampleMean))
### Compute the test statistic.
TestStat = (SampleMean-TrueMean)/(TrueStd/np.sqrt(SampleSize))
### Compute the probability value (p value).
ProbVal = 2*scipy.stats.norm.cdf(-np.abs(TestStat))
print('* Test statistic    : {:3f}'.format(TestStat))
print('* Probability value : {:3f}'.format(ProbVal))
### Check how the p value compares with the significance level alpha.
if ProbVal >= alpha:
    print('No significant difference between sample mean and true mean.')
else:
    print('Statistically significant difference between sample mean and true mean.')

Since in this case the sample is drawn from the stated underlying distribution, we expect that in most cases (confidence level $\gamma = 1 - \alpha$) the null hypothesis is not rejected, i.e., the sample mean does not differ significantly from the true mean. To check this expectation, we repeat the procedure using a large set of samples, and plot the distribution of $p$ values.

In [ ]:
### Define the true parameters of the underlying normal distribution.
TrueMean = 3
TrueStd = 5
### Initialize arrays to store statistics of the ensemble of samples.
N = SampleSize = 100
NumSamples = 1000
SampleMeanArr = np.zeros(NumSamples)
TestStatArr = np.zeros(NumSamples)
ProbValArr = np.zeros(NumSamples)
### Create arrays with sample statistics and test statistics.
for k in range(NumSamples):
    data = scipy.stats.norm.rvs(loc=TrueMean,scale=TrueStd,size=SampleSize)
    SampleMeanArr[k] = np.mean(data)
    TestStatArr[k] = (SampleMeanArr[k]-TrueMean)/(TrueStd/np.sqrt(SampleSize))
    ProbValArr[k] = 2*scipy.stats.norm.cdf(-np.abs(TestStatArr[k]))
### Open plot figure.
plt.figure(figsize=(11,4))
### Plot distribution of test statistics.
plt.subplot(1,2,1)
epdf,bins,pats = plt.hist(TestStatArr,20,density=True)
plt.plot(bins,scipy.stats.norm.pdf(bins),color='orange')
plt.title('Distribution of test statistics')
plt.xlabel(r'Test statistic $z$')
plt.ylabel('Normed PDF')
zCrit = scipy.stats.norm.ppf(alpha/2)
plt.plot([zCrit,zCrit],[0,scipy.stats.norm.pdf(zCrit)],color='red')
plt.plot([-zCrit,-zCrit],[0,scipy.stats.norm.pdf(-zCrit)],color='red')
### Plot distribution of probability values.
plt.subplot(1,2,2)
plt.hist(ProbValArr,20)
ylimits = plt.gca().get_ylim()
plt.plot([alpha,alpha],ylimits,color='red')
plt.xlim([0,1])
plt.ylim(ylimits)
plt.title(r'Distribution of $p$ values (two-tailed $z$ test)')
plt.xlabel('Binned $p$ values')
plt.ylabel('Occurence frequency')

The left diagram shows the empirical (blue histogram) and theoretical (orange line) distributions of the test statistic $z$. The two rejection regions are marked by two vertical red lines. The right diagram displays the (supposedly uniform) empirical distribution of $p$ values. Here the vertical red line gives the signficance level. The region to the left of the red line contains the cases where the sample means are so extreme that the resulting $p$ values are below the pre-selected significance level $\alpha$, and the null hypothesis $H_0$ is erroneously rejected (population of false positives).

### Exercise: $z$ test

Repeat the procedure using random variates from a standard uniform distribution on the interval $[0,1]$. Note the mean is $\mu=\frac{1}{2}$, and the variance is $\sigma^2 = \frac{1}{12}$.

### Student's $t$ test

This category summarizes statistical hypothesis tests where under the null hypothesis $H_0$, the test statistic 
$$
T \, = \, \frac{\bar{u}-\mu}{\sigma/\sqrt{N}}
$$
follows a Student's $t$ distribution, e.g., when the sample mean $\bar{u}$ is compared with the known true mean $\mu$ but the standard deviation is unknown, and the sample size $N$ is small. Under the null hypothesis $H_0$ it is assumed that the sample mean $\bar{u}$ does not show a significant difference from the true mean $\mu$.

To demonstrate this test, we repeat the numerical $z$ test exercise with a small sample size. 

In [ ]:
## Define the true parameters of the underlying normal distribution.
TrueMean = 3
print('True mean           : {:3f}'.format(TrueMean))
TrueStd = 5
### Obtain a sample drawn from the underlying normal distribution.
N = SampleSize = 10
print('Sample size         : {}'.format(SampleSize))
data = scipy.stats.norm.rvs(loc=TrueMean,scale=TrueStd,size=SampleSize)
### Compute sample statistics.
SampleMean = np.mean(data)
SampleStd = np.std(data,ddof=1)
print('Sample mean         : {:3f}'.format(SampleMean))
print('Sample std          : {:3f}'.format(SampleStd))
### Compute test statistic.
TestStat = (SampleMean-TrueMean)/(SampleStd/np.sqrt(SampleSize))
### Compute the probability value (p value).
ProbVal = 2*scipy.stats.t.cdf(-np.abs(TestStat),df=SampleSize-1)
print('* Test statistic    : {:3f}'.format(TestStat))
print('* Probability value : {:3f}'.format(ProbVal))
### Check how the p value compares with the significance level alpha.
if ProbVal >= alpha:
    print('No significant difference between sample mean and true mean.')
else:
    print('Statistically significant difference between sample mean and true mean.')

This test is implemented in the function ``ttest_1samp()`` of the ``scipy.stats`` module.

In [ ]:
### Compute the test statistic and the probability value (p value).
TestStat,ProbVal = scipy.stats.ttest_1samp(data,TrueMean)
print('Test statistic    : {:3f}'.format(TestStat))
print('Probability value : {:3f}'.format(ProbVal))

### Exercise: Two-sample $t$ test

Consult the literature on the two-sample $t$ test, and then the documentation of ``scipy.stats.ttest_ind()`` to learn how the test is implemented. Following the examples above, generate two samples of random variates and demonstrate the usage of the function ``ttest_ind()``.

### Normality tests

Several normality tests are implemented in ``scipy.stats``. D'Agostino's $K^2$ test is based on the sample skewness and the sample kurtosis. In the Anderson-Darling test, the test statistic is constructed using the empirical CDF. The Shapiro-Wilk test is related to the normal Q-Q (quantile-quantile) plot, see below.

In [ ]:
N = SampleSize = 100
print('Sample size         : {}'.format(SampleSize))
#.. Generate a sample drawn from a normal distribution.
data1 = scipy.stats.norm.rvs(size=SampleSize)
TestStat,ProbVal = scipy.stats.shapiro(data1)
print('\nFirst sample (normal distribution)')
print('* Test statistic    : {:3f}'.format(TestStat))
print('* Probability value : {:3f}'.format(ProbVal))
if ProbVal >= alpha:
    print('No support for rejecting the normality hypothesis.')
else:
    print('Reject the normality hypothesis.')
#.. Generate a sample drawn from a uniform distribution.
data2 = scipy.stats.uniform.rvs(size=SampleSize)
TestStat,ProbVal = scipy.stats.shapiro(data2)
print('\nSecond sample (uniform distribution)')
print('* Test statistic    : {:3f}'.format(TestStat))
print('* Probability value : {:3f}'.format(ProbVal))
if ProbVal >= alpha:
    print('No support for rejecting the normality hypothesis.')
else:
    print('Reject the normality hypothesis.')

### Exercise: Normality tests

Repeat this exercise using D'Agostino's $K^2$ test as implemented in `scipy.stats.normaltest()`.

### Normal probability plot

A graphical means to check normality is the normal probability (Q-Q, quantile-quantile) plot. See the documentation on the function `probplot()` for further information.

In [ ]:
N = SampleSize = 100
print('Sample size         : {}'.format(SampleSize))
### Generate a sample drawn from a normal distribution.
data1 = scipy.stats.norm.rvs(size=SampleSize)
### Generate a sample drawn from a uniform distribution.
data2 = scipy.stats.uniform.rvs(size=SampleSize)
### Open plot figure
plt.figure(figsize=(11,4))
### Normal probability plot for the first sample.
plt.subplot(1,2,1)
scipy.stats.probplot(data1,plot=plt,fit=True)
plt.title('Normal probability plot for the 1st sample')
plt.xlabel('Theoretical quantiles (standard Gaussian)')
plt.ylabel('Empirical quantiles (ordered data)')
plt.grid()
### Normal probability plot for the second sample.
plt.subplot(1,2,2)
scipy.stats.probplot(data2,plot=plt,fit=True)
plt.title('Normal probability plot for the 2nd sample')
plt.xlabel('Theoretical quantiles (standard Gaussian)')
plt.ylabel('Empirical quantiles (ordered data)')
plt.grid()

The linear pattern in the normal probability plot on the left offers strong evidence for the normality hypothesis, and the linear correlation coefficient is very close to one. The intercept of the regression line gives an estimate of the mean, and the slope an estimate of the standard deviation. In the diagram on the right, the probability plot displays a nonlinear pattern, particularly away from the center, with the first few points above the line and the last few points below the line (S-shaped pattern), indicative of a symmetric distribution with short tails.

### Exercise: Normal probability plot

Consult the section [Normal Probability Plot](https://www.itl.nist.gov/div898/handbook/eda/section3/normprp1.htm) of the online [NIST/SEMATECH e-Handbook of Statistical Methods](http://www.itl.nist.gov/div898/handbook/) to learn how to interpret normal probability plots.

---
---

## Solutions

### Solution: Density estimation

In [ ]:
rng = np.random.default_rng(99)
u = rng.random(1000)
x = np.array([-0.5,-0.0001,0.0,1.0,1.0001,1.5])
d = np.ones(len(x))
d[x<0] = 0
d[x>1] = 0
plt.figure(figsize=(15,4))
plt.subplot(1,2,1)
plt.plot(x,d,label='Theoretical PDF')
plt.hist(u,bins=20,density=True,label='Empirical PDF')
plt.title('Uniform distribution and random numbers: histogram')
plt.xlim((-0.5,1.5))
plt.ylim((0.0,1.5))
plt.grid()
plt.legend()
plt.subplot(1,2,2)
plt.plot(x,d,label='Theoretical PDF')
kdeplot(u,fill=True,label='Empirical PDF')
plt.title('Uniform distribution and random numbers: KDE plot')
plt.xlim((-0.5,1.5))
plt.ylim((0.0,1.5))
plt.grid()
plt.legend()

### Solution: Normal distribution in `scipy.stats`

In [ ]:
### Standard normal distribution
print('Normal distribution: mean=0, std=1')
nrv1 = scipy.stats.norm()
print('* Support:',nrv1.support())
m,v,s,k = nrv1.stats(moments='mvsk')
print('* Moments: m={:.2f}, v={:.4f}, s={:.2f}, k={:.3f}'.format(m,v,s,k))
print()
### Normal distribution with 
print('Normal distribution: mean=-1, std=2')
uloc = -1
usca = 2
nrv2 = scipy.stats.norm(loc=uloc,scale=usca)
print('* Support:',nrv2.support())
m,v,s,k = nrv2.stats(moments='mvsk')
print('* Moments: m={:.2f}, v={:.4f}, s={:.2f}, k={:.3f}'.format(m,v,s,k))

In [ ]:
x = np.linspace(-7,5,121)
plt.figure(figsize=(15,4))
plt.subplot(131)
plt.title('Probability density functions')
plt.plot(x,nrv1.pdf(x),'b-',lw=3,label='Standard normal')
plt.plot(x,nrv2.pdf(x),'r-',lw=3,label='loc={:}, scale={:}'.format(uloc,usca))
plt.grid()
plt.legend()
plt.subplot(132)
plt.title('Cumulative distribution functions')
plt.plot(x,nrv1.cdf(x),'b-',lw=3,label='Standard normal')
plt.plot(x,nrv2.cdf(x),'r-',lw=3,label='loc={:}, scale={:}'.format(uloc,usca))
plt.legend()
plt.grid()
plt.subplot(133)
q = np.linspace(0,1,101)
plt.title('Percent point (quantile) functions')
plt.plot(q,nrv1.ppf(q),'b-',lw=3,label='Standard normal')
plt.plot(q,nrv2.ppf(q),'r-',lw=3,label='loc={:}, scale={:}'.format(uloc,usca))
plt.legend()
plt.grid()

### Solution: $z$ test

In [ ]:
### Define the true parameters of the underlying uniform distribution.
TrueMean = 0.5
TrueStd = 1/np.sqrt(12)
### Initialize arrays to store statistics of the ensemble of samples.
N = SampleSize = 100
NumSamples = 1000
SampleMeanArr = np.zeros(NumSamples)
TestStatArr = np.zeros(NumSamples)
ProbValArr = np.zeros(NumSamples)
### Create arrays with sample statistics and test statistics.
for k in range(NumSamples):
    data = scipy.stats.uniform.rvs(size=SampleSize)
    SampleMeanArr[k] = np.mean(data)
    TestStatArr[k] = (SampleMeanArr[k]-TrueMean)/(TrueStd/np.sqrt(SampleSize))
    ProbValArr[k] = 2*scipy.stats.norm.cdf(-np.abs(TestStatArr[k]))
### Open plot figure.
plt.figure(figsize=(11,4))
### Plot distribution of test statistics.
plt.subplot(1,2,1)
epdf,bins,pats = plt.hist(TestStatArr,20,density=True)
plt.plot(bins,scipy.stats.norm.pdf(bins),color='orange')
plt.title('Distribution of test statistics')
plt.xlabel(r'Test statistic $z$')
plt.ylabel('Normed PDF')
zCrit = scipy.stats.norm.ppf(alpha/2)
plt.plot([zCrit,zCrit],[0,scipy.stats.norm.pdf(zCrit)],color='red')
plt.plot([-zCrit,-zCrit],[0,scipy.stats.norm.pdf(-zCrit)],color='red')
### Plot distribution of probability values.
plt.subplot(1,2,2)
plt.hist(ProbValArr,20)
ylimits = plt.gca().get_ylim()
plt.plot([alpha,alpha],ylimits,color='red')
plt.xlim([0,1])
plt.ylim(ylimits)
plt.title(r'Distribution of $p$ values (two-tailed $z$ test)')
plt.xlabel('Binned $p$ values')
plt.ylabel('Occurence frequency')

### Solution: Two-sample $t$ test

In [ ]:
### Define the true parameters of the underlying normal distributions.
TrueMean1 = 3
print('True mean (1)     : {:3f}'.format(TrueMean1))
TrueStd1 = 5
TrueMean2 = TrueMean1
print('True mean (2)     : {:3f}'.format(TrueMean2))
TrueStd2 = TrueStd1
### Obtain two samples drawn from the underlying normal distributions.
N = SampleSize = 10
print('Sample size       : {}'.format(SampleSize))
data1 = scipy.stats.norm.rvs(loc=TrueMean1,scale=TrueStd1,size=SampleSize)
SampleMean1 = np.mean(data1)
print('Sample mean (1)   : {:3f}'.format(SampleMean1))
data2 = scipy.stats.norm.rvs(loc=TrueMean2,scale=TrueStd2,size=SampleSize)
SampleMean2 = np.mean(data2)
print('Sample mean (2)   : {:3f}'.format(SampleMean2))
### Compute the test statistic and the probability value (p value).
TestStat,ProbVal = scipy.stats.ttest_ind(data1,data2)
print('* Test statistic    : {:3f}'.format(TestStat))
print('* Probability value : {:3f}'.format(ProbVal))
### Check how the p value compares with the significance level alpha.
if ProbVal >= alpha:
    print('No significant difference between the sample means.')
else:
    print('Significant difference between the sample means.')

### Solution: Normality tests

In [ ]:
N = SampleSize = 100
print('Sample size         : {}'.format(SampleSize))
#.. Generate a sample drawn from a normal distribution.
data1 = scipy.stats.norm.rvs(size=SampleSize)
TestStat,ProbVal = scipy.stats.normaltest(data1)
print('\nFirst sample (normal distribution)')
print('* Test statistic    : {:3f}'.format(TestStat))
print('* Probability value : {:3f}'.format(ProbVal))
if ProbVal >= alpha:
    print('No support for rejecting the normality hypothesis.')
else:
    print('Reject the normality hypothesis.')
#.. Generate a sample drawn from a uniform distribution.
data2 = scipy.stats.uniform.rvs(size=SampleSize)
TestStat,ProbVal = scipy.stats.normaltest(data2)
print('\nSecond sample (uniform distribution)')
print('* Test statistic    : {:3f}'.format(TestStat))
print('* Probability value : {:3f}'.format(ProbVal))
if ProbVal >= alpha:
    print('No support for rejecting the normality hypothesis.')
else:
    print('Reject the normality hypothesis.')

---
---